In [36]:
using JLD2
using Flux
using MLJ
using MLJLIBSVMInterface

include("../src/filters.jl")
include("../src/reduction.jl")

@JLD2.load "variables/train_dataset.jld2" train_inputs train_targets
@JLD2.load "variables/test_dataset.jld2" test_inputs test_targets

2-element Vector{Symbol}:
 :test_inputs
 :test_targets

In [13]:
function buildANN(n_inputs::Int, hidden_layers::Vector{Int}, n_outputs::Int; activation = relu)
    layers = Any[]

    # capas ocultas
    in_dim = n_inputs
    for h in hidden_layers
        push!(layers, Dense(in_dim, h, activation))
        in_dim = h
    end

    # capa de salida
    push!(layers, Dense(in_dim, n_outputs))
    push!(layers, Flux.softmax)

    return Chain(layers...)
end

buildANN (generic function with 1 method)

In [9]:
function trainANN(ann::Chain, dataset::Tuple{DataFrame, BitArray};
    epochs::Int=1000, learningRate::Real=0.001, minLossChange::Real=1e-7, lossChangeWindowSize::Int=5)

    inputs = Matrix(dataset[1]) |> x -> Float32.(x)
    targets = Matrix(dataset[2]) |> x -> Float32.(x)
    # Flux espera columnas = muestras
    inputs = inputs'
    targets = targets'

    loss(model, x, y) = Flux.Losses.crossentropy(model(x), y);
    trainingLosses = Float32[loss(ann, inputs, targets)];
    opt_state = Flux.setup(Adam(learningRate), ann);

    for epoch in 1:epochs
        Flux.train!(loss, ann, [(inputs, targets)], opt_state);
        current_loss = Float32(loss(ann, inputs, targets));
        push!(trainingLosses, current_loss);
    end;

    return trainingLosses
end;

In [38]:
function createSVM(dataset::Tuple{DataFrame, BitArray}, kernel::String, C::Real; gamma::Real=1.0, degree::Int=3, coef0::Real=0.0)

    # deshacemos One-Hot-Encoding 
    inputs, labels = dataset; 
    targets = [sum(findall(row)) for row in eachrow(labels)]

    if kernel == "linear"
        k = LIBSVM.Kernel.Linear
    elseif kernel == "rbf"
        k = LIBSVM.Kernel.RadialBasis
    elseif kernel == "poly"
        k = LIBSVM.Kernel.Polynomial
    elseif kernel == "sigmoid"
        k = LIBSVM.Kernel.Sigmoid
    else
        error("Kernel no soportado: $kernel")
    end

    model = SVMClassifier(
        kernel = k,
        cost = Float64(C),
        gamma = Float64(gamma),
        degree = Int32(degree),
        coef0 = Float64(coef0)
    )

    mach = machine(
        model,
        inputs,
        categorical(targets)
    )

    MLJ.fit!(mach)
    return mach
end

createSVM (generic function with 2 methods)

In [6]:
function createKNN(dataset::Tuple{DataFrame, BitArray}, k::Int = 5)

    # Deshacemos One-Hot-Encoding   
    inputs, labels = dataset; 
    targets = [sum(findall(row)) for row in eachrow(labels)]

    model = KNNClassifier(K = k)

    mach = machine(
        model,
        MLJ.table(inputs),
        categorical(targets)
    )

    fit!(mach)
    return mach
end

createKNN (generic function with 2 methods)

In [ ]:
inputs, targets = train_inputs, train_targets

# 1. PCA
X_pca, pca_model = pca(inputs, 0.95)

# 2. ANOVA
anova_scores = anova((X_pca, train_targets));
anova_index = findall(x -> x >= 0.3, anova_scores);
new_dataset = (X_pca[:, anova_index], train_targets);

# 3. ANN
n_inputs = size(new_dataset[1], 2)
n_outputs = size(new_dataset[2], 2)
ann_model = buildANN(n_inputs, [50], n_outputs)
losses = trainANN(ann_model, (new_dataset[1], new_dataset[2]))



TODO bien

In [41]:
# 1. PEARSON
filtered_train_inputs, correlations = pearson((train_inputs, train_targets))
new_dataset = (filtered_train_inputs, train_targets);

# 2. LDA
X_proj, W = lda(new_dataset)
X_proj_real = real.(X_proj)      # si hay complejos, toma la parte real
X_proj_clean = coalesce.(X_proj_real, 0.0)   # rellena posibles missing con 0
X_lda = DataFrame(X_proj_clean, :auto)
new_dataset = (X_lda, train_targets)

# 3. SVM
mach = createSVM(new_dataset, "linear", 0.5)


┌ Info: Training machine(SVC(kernel = Linear, …), …).
└ @ MLJBase C:\Users\Usuario\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


trained Machine; caches model-specific representations of data
  model: SVC(kernel = Linear, …)
  args: 
    1:	Source @044 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @671 ⏎ AbstractVector{Multiclass{6}}


In [ ]:
# 1. KENDALL
kendall_coeff = kendall((train_inputs, train_targets))
kendall_index = findall(x -> x >= 0.3, kendall_coeff);
new_dataset = (train_inputs[:, kendall_index], train_targets);

# 2. ICA
S, W = fastica(new_dataset; tol=0.01, max_iter=100)
S = DataFrame(S, :auto)
new_dataset = (S, categorical(train_targets))

# 3. KNN
mach = createKNN(new_dataset; k=10)

MethodError: MethodError: no method matching createKNN(::Tuple{Matrix{Float64}, BitMatrix}; k::Int64)
The function `createKNN` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  createKNN(!Matched::Tuple{DataFrame, BitArray}, !Matched::Int64) got unsupported keyword argument "k"
   @ Main c:\Users\Usuario\Documents\adrispul\Universidad\INTELIGENCIA ARTIFICIAL\3º\1º SEMESTRE\MAAA\PRÁCTICAS\GitHub_Repository\Practicas_Julia\Practica4\notebooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:1
  createKNN(!Matched::Tuple{DataFrame, BitArray}) got unsupported keyword argument "k"
   @ Main c:\Users\Usuario\Documents\adrispul\Universidad\INTELIGENCIA ARTIFICIAL\3º\1º SEMESTRE\MAAA\PRÁCTICAS\GitHub_Repository\Practicas_Julia\Practica4\notebooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:1
